### Предобработка данных в Pandas
----

In [144]:
from pandas import DataFrame, read_csv
data = pandas.read_csv('titanic.csv', index_col='PassengerId')

Какое количество мужчин и женщин ехало на корабле? В качестве ответа приведите два числа через пробел.

In [24]:
df = data.groupby('Sex')
df.size()
# Faster would have been  to use value_counts()

Sex
female    314
male      577
dtype: int64

Какой части пассажиров удалось выжить? Посчитайте долю выживших пассажиров. Ответ приведите в процентах (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.

In [119]:
round(float(data[data['Survived'] == 1].size) / data.size * 100, 2)

38.38

Какую долю пассажиры первого класса составляли среди всех пассажиров? Ответ приведите в процентах (число в интервале от 0 до 100, знак процента не нужен), округлив до двух знаков.

In [121]:
round(float(data[data['Pclass'] == 1].size) / data.size * 100, 2)

24.24

Какого возраста были пассажиры? Посчитайте среднее и медиану возраста пассажиров. В качестве ответа приведите два числа через пробел.

In [124]:
d = data['Age']
round(d.median(), 2)
#"" + str(round(d.mean())) + " " + str(d.median())

28.0

Коррелируют ли число братьев/сестер/супругов с числом родителей/детей? Посчитайте корреляцию Пирсона между признаками SibSp и Parch.

In [61]:
from scipy.stats.stats import pearsonr

In [137]:
#corr = pearsonr(data['SibSp'], data['Parch'])
c = data[['SibSp', 'Parch']].corr(method='pearson')
round(c['SibSp']['Parch'], 2)

0.41

Какое самое популярное женское имя на корабле? Извлеките из полного имени пассажира (колонка Name) его личное имя (First Name). Это задание — типичный пример того, с чем сталкивается специалист по анализу данных. Данные очень разнородные и шумные, но из них требуется извлечь необходимую информацию. Попробуйте вручную разобрать несколько значений столбца Name и выработать правило для извлечения имен, а также разделения их на женские и мужские.

In [116]:
names = data[data.Sex == 'female'].Name.str.extract(r'\((\w+)', expand=True)
names[1] = data[data.Sex == 'female'].Name.str.extract(r'[Miss|Mrs]\. (\w+)', expand=False)
names[2] = names[0].combine_first(names[1])
gb = names.groupby(names[2]).size()
gb.name = 'size'
gb = gb.reset_index()
gb.sort_values(by='size', ascending=False).head(1)

,2,size
13,Anna,15


### Важность признаков
----

Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).

In [166]:
data['Gender'] = ((data['Sex'] == 'male')).astype('int')
data1 = data.dropna()

Обратите внимание, что признак Sex имеет строковые значения.
Выделите целевую переменную — она записана в столбце Survived.

In [167]:
target = data1['Survived']

В данных есть пропущенные значения — например, для некоторых пассажиров неизвестен их возраст. Такие записи при чтении их в pandas принимают значение nan. Найдите все объекты, у которых есть пропущенные признаки, и удалите их из выборки.

Обучите решающее дерево с параметром random_state=241 и остальными параметрами по умолчанию (речь идет о параметрах конструктора DecisionTreeСlassifier).

In [199]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
#X = np.array([[1, 2], [3, 4], [5, 6]])
#y = np.array([0, 1, 0])
clf = DecisionTreeClassifier(random_state=241)
clf.fit(data1[['Fare', 'Gender', 'Age', 'Pclass']], data1['Survived'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=241, splitter='best')

Вычислите важности признаков и найдите два признака с наибольшей важностью. Их названия будут ответами для данной задачи (в качестве ответа укажите названия признаков через запятую или пробел, порядок не важен).

In [200]:
importances = clf.feature_importances_

In [201]:
importances #The right answer is Fare Sex(Gender)

array([ 0.29762263,  0.2834689 ,  0.4009751 ,  0.01793336])

In [171]:
data1.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Gender
PassengerId,,,,,,,,,,,,
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,1
11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,0
12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,0
22,1,2,"Beesley, Mr. Lawrence",male,34.0,0,0,248698,13.0000,D56,S,1
24,1,1,"Sloper, Mr. William Thompson",male,28.0,0,0,113788,35.5000,A6,S,1
28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S,1
53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C,0
